In [ ]:
! pip install yfinance

In [ ]:
import yfinance as yf
data = yf.download("AAPL IBM", start="2009-01-01", end="2019-12-31")

[*********************100%***********************]  2 of 2 completed


In [ ]:
data['Open']

,AAPL,IBM
Date,,
2009-01-02,3.067143,83.889999
2009-01-05,3.327500,86.419998
2009-01-06,3.426786,87.110001
2009-01-07,3.278929,87.830002
2009-01-08,3.229643,87.809998
...,...,...
2019-12-23,70.132500,135.779999
2019-12-24,71.172501,135.610001
2019-12-26,71.205002,134.979996


In [ ]:
%%bash
$HADOOP_HOME/sbin/start-dfs.sh
$HADOOP_HOME/sbin/start-yarn.sh
$HADOOP_HOME/bin/mapred --daemon start historyserver

Starting namenodes on [localhost]
Starting datanodes
Starting secondary namenodes [p-27d0a350-1ca3-4f23-a945-8e1c7c8e623e]
Starting resourcemanager
Starting nodemanagers


In [ ]:
from hdfs3 import HDFileSystem
hdfs = HDFileSystem(host='localhost', port=9000)
with hdfs.open('AAPL_IBM_open.csv', 'wb') as f:
    data['Open'].to_csv(f,header=True)


OSError: Could not open file: AAPL_IBM_open.csv, mode: wb Cannot create file/user/jovyan/AAPL_IBM_open.csv. Name node is in safe mode.

In [ ]:
hdfs.ls('.')
hdfs.cat('./AAPL_IBM_open.csv')

b'Date,AAPL,IBM\n2009-01-02,3.067142963409424,83.88999938964844\n2009-01-05,3.327500104904175,86.41999816894531\n2009-01-06,3.426785707473755,87.11000061035156\n2009-01-07,3.278928518295288,87.83000183105469\n2009-01-08,3.229642868041992,87.80999755859375\n2009-01-09,3.3289284706115723,87.05000305175781\n2009-01-12,3.2307143211364746,84.56999969482422\n2009-01-13,3.151428461074829,85.5\n2009-01-14,3.0799999237060547,84.19999694824219\n2009-01-15,2.877500057220459,83.61000061035156\n2009-01-16,3.010714292526245,84.61000061035156\n2009-01-20,2.9260714054107666,84.76000213623047\n2009-01-21,2.835357189178467,86.29000091552734\n2009-01-22,3.1442856788635254,89.83000183105469\n2009-01-23,3.1007142066955566,88.4000015258789\n2009-01-26,3.1735713481903076,89.7699966430664\n2009-01-27,3.221071481704712,91.7699966430664\n2009-01-28,3.2899999618530273,92.69999694824219\n2009-01-29,3.3246428966522217,93.58000183105469\n2009-01-30,3.307142972946167,92.2300033569336\n2009-02-02,3.182142972946167,90

In [ ]:
%%file stock_analysis_spark.py

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

import re
import sys

spark = SparkSession.builder.appName("Stock Analysis").getOrCreate()

mySchema = StructType([
    StructField("date", DateType(), False),
    StructField("apple_value", DoubleType(), False),
    StructField("ibm_value", DoubleType(), False)])

df = spark.read.format('csv').option("sep", ",").schema(mySchema).load("AAPL_IBM_open.csv")
df.show()

df.filter(data("date").lt(lit("2015-03-14")))
apple_stock = df.groupBy("date").agg(max("apple_value"))
ibm_stock = df.groupBy("date").agg(max("ibm_value"))

apple_stock.write.format("csv").option("sep", "\t").option("path", "output").save()
ibm_stock.write.format("csv").option("sep", "\t").option("path", "output").save()

Overwriting stock_analysis_spark.py


In [ ]:
! $SPARK_HOME/bin/spark-submit --master yarn stock_analysis_spark.py

2020-11-23 11:19:44,380 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2020-11-23 11:19:45,295 INFO spark.SparkContext: Running Spark version 3.0.1
2020-11-23 11:19:45,372 INFO resource.ResourceUtils: ==============================================================
2020-11-23 11:19:45,374 INFO resource.ResourceUtils: Resources for spark.driver:

2020-11-23 11:19:45,374 INFO resource.ResourceUtils: ==============================================================
2020-11-23 11:19:45,374 INFO spark.SparkContext: Submitted application: Stock Analysis
2020-11-23 11:19:45,472 INFO spark.SecurityManager: Changing view acls to: jovyan
2020-11-23 11:19:45,473 INFO spark.SecurityManager: Changing modify acls to: jovyan
2020-11-23 11:19:45,473 INFO spark.SecurityManager: Changing view acls groups to: 
2020-11-23 11:19:45,473 INFO spark.SecurityManager: Changing modify acls groups to: 
2020-11-23 11:19:45,473 INFO spark

In [ ]:
! $HADOOP_HOME/bin/hdfs dfs -get output .

get: `output/_SUCCESS': File exists
get: `output/part-00000-aed8975d-23a5-46e4-8ecd-5731db5048cd-c000.csv': File exists
get: `output/part-00001-aed8975d-23a5-46e4-8ecd-5731db5048cd-c000.csv': File exists
get: `output/part-00002-aed8975d-23a5-46e4-8ecd-5731db5048cd-c000.csv': File exists
get: `output/part-00003-aed8975d-23a5-46e4-8ecd-5731db5048cd-c000.csv': File exists
get: `output/part-00004-aed8975d-23a5-46e4-8ecd-5731db5048cd-c000.csv': File exists
get: `output/part-00005-aed8975d-23a5-46e4-8ecd-5731db5048cd-c000.csv': File exists
get: `output/part-00006-aed8975d-23a5-46e4-8ecd-5731db5048cd-c000.csv': File exists
get: `output/part-00007-aed8975d-23a5-46e4-8ecd-5731db5048cd-c000.csv': File exists
get: `output/part-00008-aed8975d-23a5-46e4-8ecd-5731db5048cd-c000.csv': File exists
get: `output/part-00009-aed8975d-23a5-46e4-8ecd-5731db5048cd-c000.csv': File exists
get: `output/part-00010-aed8975d-23a5-46e4-8ecd-5731db5048cd-c000.csv': File exists
get: `output/part-00011-aed8975d-23a5-46

In [ ]:
%%bash
$HADOOP_HOME/bin/mapred --daemon stop historyserver
$HADOOP_HOME/sbin/stop-yarn.sh
$HADOOP_HOME/sbin/stop-dfs.sh
